# Get Model Predictions of Fantasy Points 

## Get Most recent draft picks

In [2]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import string 
import time 
import nfl_data_py as nfl
import re
import lxml

In [7]:
years = [2025]

In [ ]:
draft_2025 = nfl.import_draft_picks(years)

,season,round,pick,team,gsis_id,pfr_player_id,cfb_player_id,pfr_player_name,hof,position,...,pass_ints,rush_atts,rush_yards,rush_tds,receptions,rec_yards,rec_tds,def_solo_tackles,def_ints,def_sacks
12413,2025,1,1,TEN,None,WardCa00,cameron-ward-1,Cam Ward,False,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12414,2025,1,2,JAX,None,HuntTr00,travis-hunter-2,Travis Hunter,False,WR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12415,2025,1,3,NYG,None,CartAb00,abdul-carter-1,Abdul Carter,False,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12416,2025,1,4,NWE,None,CampWi01,will-campbell-1,Will Campbell,False,OT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12417,2025,1,5,CLE,None,GrahMa00,mason-graham-2,Mason Graham,False,DT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12665,2025,7,253,MIA,None,BiggZe01,zeek-biggers-1,Zeek Biggers,False,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12666,2025,7,254,NOR,None,DiggFa00,fadil-diggs-1,Fadil Diggs,False,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12667,2025,7,255,HOU,None,LachLu00,luke-lachey-1,Luke Lachey,False,TE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12668,2025,7,256,LAC,None,BridTr00,trikweze-bridges-1,Trikweze Bridges,False,SAF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Get Drafted College Football Players from most recent draft 

draft_2025 = pd.read_csv("2025_draft.csv")
draft_2025["draft_year"] = 2025

print(len(draft_2025))
print(draft_2025[:5])

258
   Rnd  Pick   Tm         Player Pos   Age  To  AP1   PB   St  ...  TD.1  Rec  \
0  1.0   1.0  TEN       Cam Ward  QB  23.0 NaN  0.0  0.0  0.0  ...   NaN  NaN   
1  1.0   2.0  JAX  Travis Hunter  WR  22.0 NaN  0.0  0.0  0.0  ...   NaN  NaN   
2  1.0   3.0  NYG   Abdul Carter  DE  21.0 NaN  0.0  0.0  0.0  ...   NaN  NaN   
3  1.0   4.0  NWE  Will Campbell  OT  21.0 NaN  0.0  0.0  0.0  ...   NaN  NaN   
4  1.0   5.0  CLE   Mason Graham  DT  22.0 NaN  0.0  0.0  0.0  ...   NaN  NaN   

   Yds.2  TD.2  Solo  Int.1  Sk  College/Univ    Unnamed: 28  draft_year  
0    NaN   NaN   NaN    NaN NaN    Miami (FL)  College Stats        2025  
1    NaN   NaN   NaN    NaN NaN      Colorado  College Stats        2025  
2    NaN   NaN   NaN    NaN NaN      Penn St.  College Stats        2025  
3    NaN   NaN   NaN    NaN NaN           LSU  College Stats        2025  
4    NaN   NaN   NaN    NaN NaN      Michigan  College Stats        2025  

[5 rows x 30 columns]


In [5]:
# Filter for relevant positions
positions = ['QB', 'TE', 'WR', 'RB']

draft_2025 = draft_2025[draft_2025["Pos"].isin(positions)]
print(len(draft_2025))

86


In [65]:
# Add draft info to master df 
drafts_df = draft_2025
df = pd.DataFrame()
df["name"], df["position"], df["team"], df["draft_year"], df["draft_round"], df["draft_pick_overall"], df["age_on_draft_day"] = drafts_df["Player"], drafts_df["Pos"], drafts_df["Tm"], drafts_df["draft_year"], drafts_df["Rnd"], drafts_df["Pick"], drafts_df["Age"]
print(df[:2])

            name position team  draft_year  draft_round  draft_pick_overall  \
0       Cam Ward       QB  TEN        2025          1.0                 1.0   
1  Travis Hunter       WR  JAX        2025          1.0                 2.0   

   age_on_draft_day  
0              23.0  
1              22.0  


In [19]:
def get_player_id_map(names: list[str], years: list[int]) -> dict:
    """
    Build a mapping of player_id -> 'First Last' given player names and corresponding years.
    
    """
    if len(names) != len(years):
        raise ValueError("names and years must have the same length")
    
    player_map = {}
    
    # Loop through each (name, year) pair
    for name, year in zip(names, years):
        roster = nfl.import_seasonal_rosters([year])
        #roster["player_name"] = roster["first_name"] + " " + roster["last_name"]
        print(roster["height"])
        match = roster[roster["player_name"] == name]
        
        if not match.empty:
            player_id = match.iloc[0]["player_id"]
            player_map[player_id] = {
                "name": name,
                "rookie_year": year,
                "position": match.iloc[0]["position"],
                "team": match.iloc[0]["team"],
                "height_in": match.iloc[0].get("height"),
                "weight_lb": match.iloc[0].get("weight"),
                "age_on_draft_day": match.iloc[0].get("age_on_draft_day")
            }
        else:
            print(f"No match found for {name} in {year}")
    
    return player_map


In [20]:
player_map = get_player_id_map(df["name"], df["draft_year"])

0       74.0
1       73.0
2       74.0
3       78.0
4       80.0
        ... 
3035    72.0
3036    75.0
3037    75.0
3038    71.0
3039    71.0
Name: height, Length: 3040, dtype: float64
0       74.0
1       73.0
2       74.0
3       78.0
4       80.0
        ... 
3035    72.0
3036    75.0
3037    75.0
3038    71.0
3039    71.0
Name: height, Length: 3040, dtype: float64
0       74.0
1       73.0
2       74.0
3       78.0
4       80.0
        ... 
3035    72.0
3036    75.0
3037    75.0
3038    71.0
3039    71.0
Name: height, Length: 3040, dtype: float64
0       74.0
1       73.0
2       74.0
3       78.0
4       80.0
        ... 
3035    72.0
3036    75.0
3037    75.0
3038    71.0
3039    71.0
Name: height, Length: 3040, dtype: float64
0       74.0
1       73.0
2       74.0
3       78.0
4       80.0
        ... 
3035    72.0
3036    75.0
3037    75.0
3038    71.0
3039    71.0
Name: height, Length: 3040, dtype: float64
0       74.0
1       73.0
2       74.0
3       78.0
4       80.0
     

# Get College Stats

In [9]:
API_KEY = "jhx47mf3+GY8ZsAD51GHBMi7THo3VtKVg6MXZ+AkXNsvS+MoM0/5qz+F3N5hjS7B"
BASE_URL = "https://api.collegefootballdata.com"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

In [10]:
def get_player_season_stats(year: int) -> pd.DataFrame:
    """Pull season-level stats for all players in a given year."""
    url = f"{BASE_URL}/stats/player/season"
    params = {"year": year}
    try:
        # Add a small delay to avoid rate limiting
        time.sleep(0.5)
        
        r = requests.get(url, headers=HEADERS, params=params)
        r.raise_for_status()
        data = r.json()

        if not data:  # Check if data is empty
            print(f"No data returned for year {year}")
            return pd.DataFrame()
            
        df = pd.DataFrame(data)
        
    except requests.exceptions.RequestException as e:
        print(f"Request failed for year {year}: {e}")
        return pd.DataFrame()

    if df.empty:
        return pd.DataFrame()
    
    print(f"Raw columns for {year}: {df.columns.tolist()}")
    print(f"First few rows:\n{df.head()}")
    print(f"Sample statType values: {df['statType'].unique()[:10] if 'statType' in df.columns else 'No statType column'}")
    
    # Flatten schema
    df = df.rename(columns={"player": "player_name", "playerId": "player_id"})
    # Keep only columns we need
    required_cols = ["season", "player_id", "player_name", "position", "team", "conference", "category", "statType", "stat"]
    available_cols = [col for col in required_cols if col in df.columns]
    df = df[available_cols]

    # Pivot statType -> columns
    # Create a combined column for category + statType to make variables that make sense
    if "category" in df.columns and "statType" in df.columns:
        df['stat_name'] = df['category'].astype(str) + '_' + df['statType'].astype(str)
        
        print(f"Created {df['stat_name'].nunique()} unique stat combinations")
        print(f"Sample stat names: {sorted(df['stat_name'].unique())[:15]}")

        index_cols = [col for col in ["season", "player_id", "player_name", "position", "team", "conference"] if col in df.columns]
        
        print(f"Pivoting with index_cols: {index_cols}")
        print(f"Unique statTypes: {df['statType'].unique()}")
        
        df = df.pivot_table(
            index=index_cols,
            columns="stat_name",
            values="stat",
            aggfunc="sum"
        ).reset_index()

        # Convert stats to numeric
        stat_cols = df.columns.difference(index_cols)
        df[stat_cols] = df[stat_cols].apply(pd.to_numeric, errors="coerce")

        print(f"After pivot: {len(df.columns)} total columns")
        print(f"Stat columns created: {len(stat_cols)}")
        
    else:
        print("Missing statType or stat columns - cannot pivot")

    return df

In [11]:
def filter_for_fantasy_scoring(df):
    # Essential stats for fantasy point calculation
    essential_stats = [
        'season', 'player_id', 'player_name', 'position', 'team', 'conference',
        'passing_TD', 'passing_YDS', 'passing_INT',
        'rushing_TD', 'rushing_YDS',
        'receiving_REC', 'receiving_TD', 'receiving_YDS',
        'fumbles_LOST'
    ]
    
    # Optional contextual stats
    contextual_stats = [
        'passing_ATT', 'passing_COMPLETIONS', 'passing_PCT', 'passing_YPA',
        'rushing_CAR', 'rushing_YPC', 'rushing_LONG',
        'receiving_YPR', 'receiving_LONG',
        'fumbles_FUM'
    ]
    
    # Combine and filter to available columns
    all_desired_stats = essential_stats + contextual_stats
    available_cols = [col for col in all_desired_stats if col in df.columns]
    
    return df[available_cols]

In [12]:
def get_player_usage(year: int) -> pd.DataFrame:
    """Pull player usage data for a given year."""
    url = f"{BASE_URL}/player/usage"
    params = {"year": year}
    r = requests.get(url, headers=HEADERS, params=params)
    r.raise_for_status()
    df = pd.DataFrame(r.json())
    
    if df.empty:
        return pd.DataFrame()

    # Flatten usage dict
    usage_df = pd.json_normalize(df.to_dict(orient="records"), sep="_")
    usage_df = usage_df.rename(columns={"id": "player_id"})
    
    possible_usage_cols = ["player_id", "season", "usage_overall", "usage_rush", "usage_pass"]
    usage_cols = [col for col in possible_usage_cols if col in usage_df.columns]
    usage_df = usage_df[usage_cols]

    return usage_df


In [13]:
def get_college_stats_for_years(years, player_list=None):
    """
    Fetch and combine college stats for multiple years.
    """
    all_stats = []
    
    for year in years:
        print(f"Fetching college stats for year {year}...")
        
        try:
            # Get season stats
            season_stats = get_player_season_stats(year)
            if season_stats.empty:
                print(f"No season stats found for {year}")
                continue
                
            # Get usage stats
            usage_stats = get_player_usage(year)
            if usage_stats.empty:
                print(f"No usage stats found for {year}")
                # Continue with just season stats
                combined_df = season_stats.copy()
            else:
                # Merge season and usage stats
                combined_df = season_stats.merge(
                    usage_stats,
                    on=["player_id", "season"],
                    how="left"
                )
            
            combined_df["college_year"] = year
            all_stats.append(combined_df)
            
        except Exception as e:
            print(f"Error fetching data for year {year}: {e}")
            continue
    
    if not all_stats:
        print("No data retrieved for any year")
        return pd.DataFrame()
    
    # Combine all years
    final_df = pd.concat(all_stats, ignore_index=True)
    
    # Filter to specific players if provided
    if player_list is not None:
        final_df = final_df[final_df["player_name"].isin(player_list)].copy()
    
    # Fill missing values
    numeric_cols = final_df.select_dtypes(include=['float64', 'int64']).columns
    final_df[numeric_cols] = final_df[numeric_cols].fillna(0)
    
    return final_df

In [26]:
if __name__ == "__main__":
    # Could input a list of target players
    # Or get all players (and filter after if needed)
    years = [2024] 
    college_stats = get_college_stats_for_years(years)
    
    print(f"Retrieved data for {len(college_stats)} player-seasons")
    print(f"Columns: {college_stats.columns.tolist()}")
    print(college_stats.head())

Fetching college stats for year 2024...
Raw columns for 2024: ['season', 'playerId', 'player', 'position', 'team', 'conference', 'category', 'statType', 'stat']
First few rows:
   season playerId       player position        team conference   category  \
0    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   
1    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   
2    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   
3    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   
4    2024   102597  Will Rogers       QB  Washington    Big Ten  defensive   

  statType stat  
0       PD    0  
1   QB HUR    0  
2    SACKS    0  
3     SOLO    1  
4       TD    0  
Sample statType values: ['PD' 'QB HUR' 'SACKS' 'SOLO' 'TD' 'TFL' 'TOT' 'FUM' 'LOST' 'REC']
Created 54 unique stat combinations
Sample stat names: ['defensive_PD', 'defensive_QB HUR', 'defensive_SACKS', 'defensive_SOLO', 'defensive_TD', 'defensive

In [27]:
# Filter for relevant columns 
filtered_college_stats = filter_for_fantasy_scoring(college_stats)

In [53]:
player_names = df["name"]
# some names in databases use different variations -- next steps would be to test name variations 
filtered_college_stats["player_name"] = filtered_college_stats["player_name"].replace("Cameron Ward", "Cam Ward")

drafted_players_college_stats = filtered_college_stats[filtered_college_stats["player_name"].isin(player_names)]

C:\Users\nloug\AppData\Local\Temp\ipykernel_28436\70848972.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_college_stats["player_name"] = filtered_college_stats["player_name"].replace("Cameron Ward", "Cam Ward")


In [54]:
print(len(drafted_players_college_stats))

79


In [55]:
# Save to .csv
college_stats.to_csv('draft_2025_college_stats.csv', index=False)
drafted_players_college_stats.to_csv('draft_2025_college_stats_filtered', index=False)

In [56]:
# Add conference strength
conference_strength_map = {
    # Power/FBS
    "SEC": 10.0,
    "Big Ten": 9.5,
    "Big 12": 8.5,
    "ACC": 8.0,
    "Pac-12": 8.0,            # historical level (adjust per season if you want)
    "American Athletic": 6.5, # AAC
    "Mountain West": 5.5,
    "Sun Belt": 5.0,
    "Mid-American": 4.5,      # MAC
    "Conference USA": 4.0,
    "FBS Independents": 6.0,  # baseline; see note below

    # FCS (top to lower)
    "MVFC": 3.6,              # Missouri Valley
    "CAA": 3.3,               # Coastal/Colonial
    "Southern": 3.2,          # SoCon
    'Big Sky': 2,
    'MEAC': 2,
    'Big South-OVC': 2,
    "OVC": 2.6,               # Ohio Valley
    "Southland": 2.5,
    "Patriot": 2.3,
    "SWAC": 2.2,
    "Ivy": 2.0,
    "Pioneer": 1.8
}

drafted_players_college_stats["conference_strength"] = drafted_players_college_stats["conference"].map(conference_strength_map)
 

C:\Users\nloug\AppData\Local\Temp\ipykernel_28436\2793275085.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafted_players_college_stats["conference_strength"] = drafted_players_college_stats["conference"].map(conference_strength_map)


In [57]:
# Add columns prefixed to distinguish college from rookie stats
drafted_players_college_stats = drafted_players_college_stats.rename(
    columns={col: f"C_{col}" for col in drafted_players_college_stats.columns if col not in ["player_id", "player_name", "position"]}
)

In [74]:
# Add height and weight from player_map 
# Convert player_map (dict of dicts) → DataFrame
player_info_df = pd.DataFrame.from_dict(player_map, orient="index").reset_index()
player_info_df = player_info_df.rename(columns={"name": "player_name"})

# Merge with rookie stats df
rookie_stats = drafted_players_college_stats
rookie_stats = rookie_stats.merge(
    player_info_df[["player_name", "height_in", "weight_lb"]],
    on="player_name",
    how="left"
)

In [75]:
# Merge 
df.rename(columns={"name": "player_name"}, inplace=True)
on_keys = ["player_name", "position"]

df_rookies_2025 = df.merge(rookie_stats, on=on_keys, how="inner")

In [76]:
print(df_rookies_2025.columns)

Index(['player_name', 'position', 'team', 'draft_year', 'draft_round',
       'draft_pick_overall', 'age_on_draft_day', 'C_season', 'player_id',
       'C_team', 'C_conference', 'C_passing_TD', 'C_passing_YDS',
       'C_passing_INT', 'C_rushing_TD', 'C_rushing_YDS', 'C_receiving_REC',
       'C_receiving_TD', 'C_receiving_YDS', 'C_fumbles_LOST', 'C_passing_ATT',
       'C_passing_COMPLETIONS', 'C_passing_PCT', 'C_passing_YPA',
       'C_rushing_CAR', 'C_rushing_YPC', 'C_rushing_LONG', 'C_receiving_YPR',
       'C_receiving_LONG', 'C_fumbles_FUM', 'C_conference_strength',
       'height_in', 'weight_lb'],
      dtype='object')


In [77]:
# Save as .csv
df_rookies_2025.to_csv('df_master_rookies_2025.csv', index=False)